**1\. In which country, game, and for which advertiser, did we have the highest IPM? Why can't we trust this number fully?**

In [ ]:
SELECT TOP 5 [country]
      ,[game_app]
      ,[advertiser]
      ,(CAST(SUM(CASE WHEN [install]=1 THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*) * 1000) AS IPM
      ,SUM(CASE WHEN [install]=1 THEN 1 ELSE 0 END) AS TotalInstalls
      , COUNT(*) AS TotalViews
  FROM [DataCleaningProjects].[dbo].[user_ad_views]
  GROUP BY [country]
      ,[game_app]
      ,[advertiser]
ORDER BY 4 DESC


We had the highest IPM with advertiser 'ndREtPhnDnoOsdsWYyvu', game app, 'eVy14GUauNxRgry1jzQD' in the US with an Install per mille of 322.58. This is not a reliable figure because the sample size is small, or the ad hasn't run for a longer period. Adverts for this particular game app has only 31 views so far.

**2\. Median and average CPI and IPM over a sliding window of** 

- **500 views (the window slides on every view)**

In [ ]:
WITH NumberedRows AS (
    SELECT
        v.[country], 
        v.[game_app], 
        a.[cpi],
        SUM(CASE WHEN v.[install]=1 THEN 1 ELSE 0 END) OVER (PARTITION BY v.[country], v.[game_app], v.[advertiser]) AS TotalInstalls,
        COUNT(*) OVER (PARTITION BY v.[country], v.[game_app], v.[advertiser]) AS TotalViews,
        ROW_NUMBER() OVER (PARTITION BY v.[country], v.[game_app] ORDER BY v.[createdat]) AS rn
    FROM [DataCleaningProjects].[dbo].[user_ad_views] v
    JOIN [DataCleaningProjects].[dbo].[advertisers_info] a
        ON v.country = a.country AND v.advertiser = a.advertiser
),
GroupedRows AS (
    SELECT
        [country],
        [game_app],
        [cpi],
        (CAST(TotalInstalls AS FLOAT) / TotalViews * 1000) AS ipm,
        rn,
        (ROW_NUMBER() OVER (PARTITION BY [country], [game_app] ORDER BY rn) - 1) / 500 + 1 AS group_number
    FROM NumberedRows
), 
Calculations AS (
    SELECT
        [country],
        [game_app],
        [cpi],
        ipm,
        AVG([cpi]) OVER (PARTITION BY country, game_app, group_number) AS avg_cpi_500_views,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY cpi ASC) over(PARTITION BY country, game_app, group_number) as median_cpi_500_views,
        AVG([ipm]) OVER (PARTITION BY country, game_app, group_number) AS avg_ipm_500_views,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY ipm ASC) over(PARTITION BY country, game_app, group_number) as median_ipm_500_views
    FROM GroupedRows
)
SELECT DISTINCT
    [country], [game_app], avg_cpi_500_views, median_cpi_500_views, avg_ipm_500_views, median_ipm_500_views
FROM Calculations
ORDER BY [country], [game_app];

**2\. Median and average CPI and IPM over a sliding window of**

- **1 day (the window slides every hour)**

In [ ]:
WITH NumberedRows AS (
    SELECT
        v.[country], 
        v.[game_app], 
        a.[cpi],
        SUM(CASE WHEN v.[install]=1 THEN 1 ELSE 0 END) OVER (PARTITION BY v.[country], v.[game_app], v.[advertiser]) AS TotalInstalls,
        COUNT(*) OVER (PARTITION BY v.[country], v.[game_app], v.[advertiser]) AS TotalViews,
        v.[createdat],
        ROW_NUMBER() OVER (PARTITION BY v.[country], v.[game_app] ORDER BY v.[createdat]) AS rn
    FROM [DataCleaningProjects].[dbo].[user_ad_views] v
    JOIN [DataCleaningProjects].[dbo].[advertisers_info] a
        ON v.country = a.country AND v.advertiser = a.advertiser
),
HourlyWindows AS (
    SELECT
        [country],
        [game_app],
        [cpi],
        (CAST(TotalInstalls AS FLOAT) / TotalViews * 1000) AS ipm,
        rn,
        v.[createdat],
        v.[createdat] - (v.[createdat] % 3600) AS window_start  -- Truncate UNIX timestamp to the nearest hour
    FROM NumberedRows v
), 
SlidingWindowCalculations AS (
    SELECT
        hw.[country],
        hw.[game_app],
        hw.[createdat],
        hw.[cpi],
        hw.[ipm],
        AVG(hw.[cpi]) OVER (PARTITION BY hw.[country], hw.[game_app], hw.[window_start]) AS avg_cpi_1day,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY hw.[cpi] ASC) over(PARTITION BY hw.[country], hw.[game_app], hw.[window_start]) as median_cpi_1day,
        AVG(hw.[ipm]) OVER (PARTITION BY hw.[country], hw.[game_app], hw.[window_start]) AS avg_ipm_1day,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY hw.[ipm] ASC) over(PARTITION BY hw.[country], hw.[game_app], hw.[window_start]) as median_ipm_1day
    FROM HourlyWindows hw
    WHERE hw.[createdat] >= hw.[window_start] - 86400  -- Consider records within the last 24 hours (86400 seconds)
)
SELECT DISTINCT
    [country], [game_app], [createdat]
        , DATEADD(second, [createdat], '1970-01-01') AS createdat_h
        , avg_cpi_1day, median_cpi_1day, avg_ipm_1day, median_ipm_1day
FROM SlidingWindowCalculations
ORDER BY [country], [game_app], [createdat];
